In [2]:
from parser import TrainingConfig, ModelConfig
from DataLoader import CustomDataLoader
training_config = TrainingConfig()
model_config = ModelConfig()

In [3]:
from collections import OrderedDict

import torch
import torch.nn as nn


class UNetSmall(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNetSmall, self).__init__()

        features = init_features
        self.encoder1 = UNetSmall._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNetSmall._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNetSmall._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bottleneck = UNetSmall._block(
            features * 4, features * 8, name="bottleneck"
        )
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNetSmall._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNetSmall._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNetSmall._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=3
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        bottleneck = self.bottleneck(self.pool3(enc3))
        dec3 = self.upconv3(bottleneck)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return torch.sigmoid(self.conv(dec1))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )
model = UNetSmall(3, 3, 32)
x = torch.rand(1, 3, 200, 200)
print(x.shape)
y = model(x)
print(y.shape)

torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 198, 198])


In [4]:
data_path = "/home/ubuntu/ml-convection/dataset"
dataloader = CustomDataLoader(data_path, training_config)
dataloader.set_required_data()
train_loader = dataloader.get_data("train")
val_loader =  dataloader.get_data("val")

In [5]:
dataloader.max_value, dataloader.min_value

([306.89, 0.304527, 0.377805], [288.495, -0.188993, -0.234059])

In [6]:
# Pretraining, sending x, getting x
import torch.nn.functional as F
import tqdm
n_epochs = 10
device = torch.device("cuda:0")
model = UNetSmall(3, 3, 32)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
for curr_epoch in range(n_epochs):
    dataloader_progress_bar = tqdm.tqdm(train_loader)
    for x, y in dataloader_progress_bar:
        optimizer.zero_grad()
        x = x.to(device)
        x_hat = model(x)
        mse_loss = F.mse_loss(x_hat, x[:, :, 1:199, 1:199])
        # add mse loss to progress bar
        dataloader_progress_bar.set_postfix(OrderedDict(mse_loss=mse_loss.item()))
        mse_loss.backward()
        optimizer.step()

100%|██████████| 50/50 [00:01<00:00, 49.73it/s, mse_loss=5.83e-5]


In [7]:
import pytorch_lightning as pl 
import torch
import torch.nn.functional as F

import numpy as np


def residual_mass(ux_matrix:np.ndarray,uy_matrix:np.ndarray):
    '''
    Compute the residual: mass conservation
    Formula: 
    Rs_mass = {d(ux)/dx + d(uy)/dy}^2.sum()/40000

    Arguments:
    ux_matrix: np.ndarray: matrix of x-velocity, shape = [200,200]
    uy_matrix: np.ndarray: matrix of y-velocity, shape = [200,200]

    Return:
    Rs_mass_sum: float: sum of Rs_mass
    '''
    ux_matrix = ux_matrix * (dataloader.max_value[1] - dataloader.min_value[1]) + dataloader.min_value[1]
    uy_matrix = uy_matrix * (dataloader.max_value[2] - dataloader.min_value[2]) + dataloader.min_value[2]
    ux_with_down_boundary = ux_matrix[:, 2:200,1:199]
    ux_with_up_boundary = ux_matrix[:, 0:198,1:199]
    uy_with_right_boundary = uy_matrix[:,1:199,2:200]
    uy_with_left_boundary = uy_matrix[:,1:199,0:198]

    pinn_dudx = (ux_with_down_boundary - ux_with_up_boundary)/(2*0.005)
    pinn_dvdy = (uy_with_right_boundary - uy_with_left_boundary)/(2*0.005)

    Rs_mass = pinn_dudx+pinn_dvdy
    Rs_mass_sq = Rs_mass*Rs_mass
    Rs_mass_sum = Rs_mass_sq.sum()/(40000*ux_matrix.shape[0])
    return Rs_mass_sum

def residual_momentum(ux_matrix, ux_matrix_prev, uy_matrix, t_matrix):
    '''
    Compute the residual: momentum conservation
    Formula:
    Rs_mom = {d(ux)/dt + ux*d(ux)/dx + uy*d(ux)/dy - 1.831e-05/(348.33/alpha)*d^2(ux)/dx^2 - 9.81/293*(293-alpha)}^2.sum()/40000
    '''
    t_matrix = t_matrix * (dataloader.max_value[0] - dataloader.min_value[0]) + dataloader.min_value[0]
    ux_matrix = ux_matrix * (dataloader.max_value[1] - dataloader.min_value[1]) + dataloader.min_value[1]
    ux_matrix_prev = ux_matrix_prev * (dataloader.max_value[1] - dataloader.min_value[1]) + dataloader.min_value[1]
    uy_matrix = uy_matrix * (dataloader.max_value[2] - dataloader.min_value[2]) + dataloader.min_value[2]
    mom_1 = ux_matrix[:,1:199,1:199] - ux_matrix_prev[:,1:199,1:199]
    mom_3 = ux_matrix[:,1:199,1:199]*(ux_matrix[:,2:200,1:199] - ux_matrix[:,0:198,1:199])
    mom_4 = uy_matrix[:,1:199,1:199]*(ux_matrix[:,1:199,2:200] - ux_matrix[:,1:199,0:198])
    mom_5_2 = ux_matrix[:,1:199,2:200] - 2*ux_matrix[:,1:199,1:199] + ux_matrix[:,1:199,0:198] 
    mom_5 = 1.831e-05/(348.33/t_matrix[:,1:199,1:199])*(mom_5_2)
    mom_6 = 9.81/293*(293-t_matrix[:,1:199,1:199])

    Rs_mom = mom_1/0.01 +  mom_3/(2*0.005) + mom_4/(2*0.005) - mom_5/(0.005*0.005) - mom_6
    Rs_mom_sq = Rs_mom*Rs_mom
    Rs_mom_sum = Rs_mom_sq.sum()/(40000*ux_matrix.shape[0])
    return Rs_mom_sum

def residual_heat(ux_matrix:np.ndarray, uy_matrix:np.ndarray, t_matrix:np.ndarray, t_matrix_prev:np.ndarray):
    '''
    Compute the residual: heat conservation
    Formula:
    Rs_heat = {d(t)/dt + ux*d(t)/dx + uy*d(t)/dy - 0.14*(t-293)+21.7/1e6*d^2(t)/dx^2}^2.sum()/40000
    TODO: Check the formula

    Arguments:
    ux_matrix: np.ndarray: matrix of x-velocity, shape = [200,200]
    uy_matrix: np.ndarray: matrix of y-velocity, shape = [200,200]
    t_matrix: np.ndarray: matrix of temperature, shape = [200,200]
    t_matrix_prev: np.ndarray: matrix of temperature at previous time step, shape = [200,200]

    Return:
    Rs_heat_sum: float: sum of Rs_heat
    '''
    tdiff_matrix = (0.14*(t_matrix[:,1:199,1:199] - 293)+ 21.7)/1000000
    heat_1 = t_matrix[:,1:199,1:199] - t_matrix_prev[:,1:199,1:199]
    heat_2 = (t_matrix[:,2:200,1:199] - t_matrix[:,0:198,1:199])*(ux_matrix[:,1:199,1:199])
    heat_3 = (t_matrix[:,1:199,2:200] - t_matrix[:,1:199,0:198])*(uy_matrix[:,1:199,1:199])
    heat_4 = tdiff_matrix*(t_matrix[:,1:199,2:200] - 2*t_matrix[:,1:199,1:199] + t_matrix[:,1:199,0:198])

    Rs_heat = heat_1/0.01 + heat_2/(2*0.005) + heat_3/(2*0.005) - heat_4/(0.005*0.005)
    Rs_heat_sq = Rs_heat*Rs_heat
    Rs_heat_sum = Rs_heat_sq.sum()/(40000* ux_matrix.shape[0])
    return Rs_heat_sum

def unnormalize_y(unnorm_arr, min_value, max_value):
    unnorm_arr[:, 0, :, :] = (
        unnorm_arr[:, 0, :, :] * (max_value[0] - min_value[0]) + min_value[0]
    )
    unnorm_arr[:, 1, :, :] = (
        unnorm_arr[:, 1, :, :] * (max_value[1] - min_value[1]) + min_value[1]
    )
    unnorm_arr[:, 2, :, :] = (
        unnorm_arr[:, 2, :, :] * (max_value[2] - min_value[2]) + min_value[2]
    )
    return unnorm_arr

In [8]:
initial_condition_matrix = torch.zeros(1, 3, 200, 200)
initial_condition_matrix[:,2,:,:] = initial_condition_matrix[:,2,:,:] + 293
initial_condition_matrix[:,2,0,:] = 288.15
initial_condition_matrix[:,2,-1,:] = 307.75
initial_condition_matrix = initial_condition_matrix.to(device)
print(initial_condition_matrix[0,2])

tensor([[288.1500, 288.1500, 288.1500,  ..., 288.1500, 288.1500, 288.1500],
        [293.0000, 293.0000, 293.0000,  ..., 293.0000, 293.0000, 293.0000],
        [293.0000, 293.0000, 293.0000,  ..., 293.0000, 293.0000, 293.0000],
        ...,
        [293.0000, 293.0000, 293.0000,  ..., 293.0000, 293.0000, 293.0000],
        [293.0000, 293.0000, 293.0000,  ..., 293.0000, 293.0000, 293.0000],
        [307.7500, 307.7500, 307.7500,  ..., 307.7500, 307.7500, 307.7500]],
       device='cuda:0')


In [ ]:

import numpy as np 
import matplotlib.pyplot as plt
true_x = initial_condition_matrix.to(device)
threshold = 0.05
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
iterations = 0
max_iterations = 1000
while True: 
    optimizer.zero_grad()
    x_hat = model(true_x)
    y_hat = torch.zeros(true_x.shape).to(device)
    y_hat[:,:2,1:199,1:199] = x_hat[:,:2,:,:]
    y_hat[:,2,0,1:199] = 288.15
    y_hat[:,2,-1,1:199] = 307.75
    y_hat[:,2,1:199,1:199] = x_hat[:,2,:,:]
    y_hat[:,2,1:199,0] = y_hat[:,2,1:199,1]
    y_hat[:,2,1:199,-1] = y_hat[:,2,1:199,-2]
    y_hat[:,2,0,0] = 0.5*(y_hat[:,2,0,1] + y_hat[:,2,1,0])
    y_hat[:,2,0,-1] = 0.5*(y_hat[:,2,0,-2] + y_hat[:,2,1,-1])
    y_hat[:,2,-1,0] = 0.5*(y_hat[:,2,-1,1] + y_hat[:,2,-2,0])
    y_hat[:,2,-1,-1] = 0.5*(y_hat[:,2,-1,-2] + y_hat[:,2,-2,-1])

    Rs_mass_sum = residual_mass(y_hat[:,0,:,:], y_hat[:,1,:,:])
    Rs_momentum_sum = residual_momentum(y_hat[:,1,:,:], true_x[:,1,:,:], y_hat[:,2,:,:], y_hat[:,0,:,:])
    Rs_heat_sum = residual_heat(y_hat[:,1,:,:], y_hat[:,2,:,:], y_hat[:,0,:,:], true_x[:,0,:,:])
    loss = Rs_mass_sum + Rs_momentum_sum + Rs_heat_sum
    loss.backward()
    optimizer.step()
    print(f"Current loss: {loss.item()}")
    if loss < threshold:
        print(f"Loss reached threshold, plotting and switching the dataset, incrementing the timestep")
        pred_y_np = x_hat.detach().cpu().numpy()
        plt.imshow(pred_y_np[0, 0, :, :])
        plt.show()
        plt.imshow(pred_y_np[0, 1, :, :])
        plt.show()
        plt.imshow(pred_y_np[0, 2, :, :])
        plt.show()
        true_x = y_hat.detach().cpu().to(device)